# 1) Enfoque personalizado

 Se considera 1 modelo por día a predecir, por tanto existen modelos que son seq-seq y seq-one. Estos modelos aprenden de secuencias que simulan exactamente la ventana de tiempo que se están considerando, por ejemplo si se quiere predecir a 3 meses 29 horarios especificos, cada uno de los Y_train de la red tienen la misma extructura y se distancian a 3 meses del X_train

Score publico de 4.1..

# Liberias 

In [ ]:
#librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import math
import numpy as np

import missingno as msno 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

# Import necessary modules
from sklearn.metrics import mean_squared_error

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from keras.callbacks import History,EarlyStopping
from tensorflow.keras.regularizers import L1, L2, L1L2

from sklearn.metrics import classification_report

import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
import pydot



#help(mp_pose.Pose)


from numpy.random import seed
import random as python_random
import random

import warnings
warnings.filterwarnings("ignore")

seed(48)
python_random.seed(48)
tf.random.set_seed(48)

#Cambiar el formato de las fechas a dd-mm-aa
import datetime
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
from sklearn.metrics import mean_squared_error

from keras.layers import LSTM
from datetime import date
from datetime import datetime, timedelta

from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input, BatchNormalization, multiply, concatenate, Flatten, Activation, dot
from keras.utils.vis_utils import plot_model, model_to_dot

In [ ]:
# hacer la conexión a Google Drive

from google.colab import drive

colab_flag = True

if (colab_flag):
    drive.mount('/content/drive',force_remount=True)



path = '/content/drive/MyDrive/ANN-2021-I/'

Mounted at /content/drive


# Preprocesamiento

In [ ]:
train_= pd.read_csv(path+'velocidad-viento/windspeed-training.csv')
train_.columns= ["Fecha","Velocidad"]
test= pd.read_csv(path+'velocidad-viento/windspeed-testing.csv')

In [ ]:
train_= pd.read_csv('velocidad-viento/windspeed-training.csv')
train_.columns= ["Fecha","Velocidad"]
test= pd.read_csv('velocidad-viento/windspeed-testing.csv')

## Imputación

In [ ]:
train=train_[(train_.Fecha< "2014-01-10" )]
fechas_imputacion=list(train_[(train_.Fecha<= "2014-01-08 23:50" )&(train_.Fecha>= "2014-01-08 16:50" )].Fecha)
reparado= [x.replace("-08","-09") for x in fechas_imputacion]
velocidad_imputacion= list(train_[(train_.Fecha<= "2014-01-08 23:50" )&(train_.Fecha>= "2014-01-08 16:40" )].Velocidad)

for i in range(len(reparado)):
    train = train.append({'Fecha': reparado[i], 'Velocidad': velocidad_imputacion[i]}, ignore_index=True)

train=train.append(train_[(train_.Fecha> "2014-01-10" )], ignore_index=True)

In [ ]:
fechas_test=list(test['26-04-2015 01:50'])
fechas_test.insert(0,'26-04-2015 01:50')
l_=pd.Series(fechas_test)
l=pd.to_datetime(l_,format='%d-%m-%Y %H:%M')
largo_seq_predecir= l.dt.date.value_counts()

In [ ]:
l_

0       26-04-2015 01:50
1       26-04-2015 03:10
2       26-04-2015 04:20
3       26-04-2015 05:20
4       26-04-2015 06:40
              ...       
1995    19-09-2015 08:10
1996    21-09-2015 11:20
1997    22-09-2015 10:30
1998    23-09-2015 17:30
1999    24-09-2015 00:10
Length: 2000, dtype: object

In [ ]:
d0 = date(2015, 4, 26)
d1 = date(2015, 9, 24)
delta = d1 - d0
print(delta.days+1)

152


In [ ]:
largo_seq_predecir

2015-06-28    29
2015-07-20    27
2015-06-15    27
2015-08-02    26
2015-05-28    25
              ..
2015-09-22     1
2015-09-17     1
2015-09-03     1
2015-09-23     1
2015-09-07     1
Length: 146, dtype: int64

# Validacion

In [ ]:
train["Fecha"] =  pd.to_datetime(train["Fecha"], format='%Y-%m-%d %H:%M')
dias_train=train.Fecha.dt.date.unique()
#para_validar=dias_train[-146:]

In [ ]:
c=0
indices_val=[]
validacion= pd.DataFrame(columns=["Fecha","Velocidad"])
for fecha in fechas_test:
  #damos vuelta el formato
  d=fecha.split("-")
  fecha_val= "2014-"+ d[1]+ "-" + d[0]+ " "+ d[2].split(" ")[1]
  #filtro
  filtro=train[train["Fecha"]== fecha_val]
  indices_val.append(filtro.index[0])
  #añadimos
  validacion.loc[c]= list(filtro.Fecha) + list(filtro.Velocidad)
  c+=1

In [ ]:
fin_entrenamiento= validacion.Fecha[0]- timedelta(minutes= 120)
entrenamiento=train[train["Fecha"]<= str(fin_entrenamiento)] 

In [ ]:
print("Conjunto de entrenamiento", train.shape)
print("Partición de validación", validacion.shape)
print("Partición de entrenamiento para validar", entrenamiento.shape)

Conjunto de entrenamiento (123407, 2)
Partición de validación (2000, 2)
Partición de entrenamiento para validar (70991, 2)


In [ ]:
año=2014 #validacion
horas_antes= 6*6 #6 horas
velocidad=list(entrenamiento.Velocidad)
entrenamiento["Fecha"] =  pd.to_datetime(entrenamiento["Fecha"], format='%Y-%m-%d %H:%M')

dias_train_=entrenamiento.Fecha.dt.date.unique()
str_fechas_entrenamiento= [str(x) for x in entrenamiento.Fecha]


In [ ]:
X_test=np.array(velocidad[-horas_antes:]).reshape(1,horas_antes)

In [ ]:
Todos_los_X_Y=[]
for i in range(largo_seq_predecir.shape[0]): #cada uno de los dias a predecir
  largo_pred= largo_seq_predecir[i]
  fecha_pred= largo_seq_predecir.index[i]
  dia_pred= fecha_pred.strftime("%Y-%m-%d").replace("2015",str(año))
  dia_sig=(fecha_pred +timedelta(days=1)).strftime("%Y-%m-%d").replace("2015",str(año))
  Y_test= validacion[(validacion["Fecha"]>= dia_pred)&(validacion["Fecha"]< dia_sig)]
  delta=(datetime.strptime(dia_pred, '%Y-%m-%d').date()-fin_entrenamiento.date()).days
  horario_exacto=[]
  for dia in Y_test.Fecha:
    horario_exacto.append(str(dia).split(" ")[1].split(":")[0]+":"+str(dia).split(" ")[1].split(":")[1])

  X_train,Y_train= generador_train(largo_pred)
  
  Todos_los_X_Y.append((X_train,Y_train,Y_test.Velocidad))

In [ ]:
def generador_train( largo_pred):
  i=0
  for d in dias_train_[1:]: #no tomo el primero
    y=[]
    for h in horario_exacto:
      consulta_y=str(d)+ " "+ h+":00"
      if consulta_y in str_fechas_entrenamiento:
        y.append(entrenamiento[entrenamiento["Fecha"]== str(d)+ " "+ h].Velocidad.values[0])
      else: 
        break 
    primer_y=entrenamiento[entrenamiento["Fecha"]==str(d)+ " "+ horario_exacto[0]].index[0]
    
    if len(y)==largo_pred and primer_y>horas_antes+120+delta*144:
      
      x=np.array(velocidad[primer_y-120-delta*144-horas_antes:primer_y-120-delta*144]).reshape(1,horas_antes)
      y= np.array(y).reshape(1,largo_pred)
      if i==0:
          i+=1
          X = x
          Y = y
      else:
          X = np.vstack((X,x))
          Y = np.vstack((Y,y))
    else:
      pass
  return X.reshape(X.shape[0], X.shape[1], 1), Y

In [ ]:
i=0
for d in dias_train_[1:]: #no tomo el primero
  y=[]
  for h in horario_exacto:
    consulta_y=str(d)+ " "+ h+":00"
    if consulta_y in str_fechas_entrenamiento:
        y.append(entrenamiento[entrenamiento["Fecha"]== str(d)+ " "+ h].Velocidad.values[0])
    else: 
      print(h)
      print(d)
      f= entrenamiento[entrenamiento["Fecha"]== str(d)+ " "+ h]
      break 
  primer_y=entrenamiento[entrenamiento["Fecha"]==str(d)+ " "+ horario_exacto[0]].index[0]
  if len(y)==largo_pred and primer_y>horas_antes+120+delta*144:
    x=np.array(velocidad[primer_y-120-delta*144-horas_antes:primer_y-120-delta*144]).reshape(1,horas_antes)
    y= np.array(y).reshape(1,largo_pred)
    if i==0:
        i+=1
        X = x
        Y = y
    else:
        X = np.vstack((X,x))
        Y = np.vstack((Y,y))
  else:
    pass

NameError: name 'horario_exacto' is not defined

In [ ]:
primer_y

9510

In [ ]:
from numpy import save

In [ ]:
save('validacion.npy', np.array(Todos_los_X_Y))

In [ ]:
data = np.load("validacion.npy",allow_pickle=True)

In [ ]:
data = np.load(path+"velocidad-viento/validacion.npy",allow_pickle=True)

In [ ]:
data= np.array(Todos_los_X_Y)

NameError: name 'Todos_los_X_Y' is not defined

In [ ]:
data.shape

(146, 3)

# Modelos

In [ ]:
X_train= data[0][0]
Y_train= data[0][1]
Y_test= data[0][2]

In [ ]:
X_train.shape

(427, 36, 1)

In [ ]:
Y_train.shape

(427, 29)

In [ ]:
Y_test.shape

(29,)

In [ ]:
X_test.shape

(1, 36)

In [ ]:
n_feautures=1

## LSTM Enconder-Decoder

In [ ]:
def enc_dec(X_train, Y_train, X_test,n_feautures=1):
    # define model
    model = Sequential()
    #enconder
    model.add(LSTM(200, activation='relu',dropout=0.2, recurrent_dropout=0.2, activity_regularizer=tf.keras.regularizers.l1(0.01),
                   kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None),input_shape=(X_train.shape[1], n_feautures)) )
    model.add(BatchNormalization())
    model.add(RepeatVector(Y_train.shape[1]))

    #decoder
    model.add(LSTM(100, activation='relu',dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    model.add(TimeDistributed(Dense(n_feautures)))

    model.compile(optimizer='adam', loss='mean_absolute_error',metrics=["mse"])

    my_callbacks = [History(), EarlyStopping(patience=3, monitor="val_loss",restore_best_weights=True)]
    model.fit(X_train, Y_train, epochs=50, verbose=0,batch_size=64, validation_split=0.2, callbacks= my_callbacks)
    x_input = X_test.reshape((1, X_train.shape[1], 1))
    ypred = model.predict(x_input, verbose=0)
    y_pred= [np.round(x[0],1) for x in ypred[0]]
    return y_pred

In [ ]:
y_pred= enc_dec(X_train, Y_train, X_test)
mean_squared_error(list(Y_test), y_pred)

Epoch 1/10
7/7 [==============================] - 8s 237ms/step - loss: 6.6339 - mse: 60.0237
Epoch 2/10
7/7 [==============================] - 2s 234ms/step - loss: 5.1699 - mse: 39.3064
Epoch 3/10
7/7 [==============================] - 2s 229ms/step - loss: 4.5414 - mse: 28.7460
Epoch 4/10
7/7 [==============================] - 2s 234ms/step - loss: 4.1939 - mse: 24.4374
Epoch 5/10
7/7 [==============================] - 2s 239ms/step - loss: 3.9319 - mse: 20.8261
Epoch 6/10
7/7 [==============================] - 2s 233ms/step - loss: 3.8327 - mse: 19.8611
Epoch 7/10
7/7 [==============================] - 2s 243ms/step - loss: 3.8046 - mse: 19.9467
Epoch 8/10
7/7 [==============================] - 2s 237ms/step - loss: 3.6892 - mse: 18.7722
Epoch 9/10
7/7 [==============================] - 2s 231ms/step - loss: 3.7625 - mse: 19.8771
Epoch 10/10
7/7 [==============================] - 2s 230ms/step - loss: 3.7259 - mse: 19.3706


22.43999985111171

## Atencional


In [ ]:
def atencionLuong(X_train, Y_train, X_test,n_feautures=1):
  input_train = Input(shape=(X_train.shape[1], n_feautures))
  output_train = Input(shape=(Y_train.shape[1],n_feautures))

    
  #primerlstm=LSTM(50, activation='elu',dropout=0.2, recurrent_dropout=0.2, activity_regularizer=tf.keras.regularizers.l1(0.01),
                #kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None), return_sequences=True)(input_train)

  #segundalstm=LSTM(50, activation='elu',dropout=0.2, recurrent_dropout=0.2, activity_regularizer=tf.keras.regularizers.l1(0.01),
                #kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None), return_sequences=True)(primerlstm)
        
  encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(200, activation='relu',dropout=0.2, recurrent_dropout=0.2, activity_regularizer=tf.keras.regularizers.l1(0.01),
                kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None),return_state=True, return_sequences=True)(input_train)

  #print(encoder_stack_h)
  #print(encoder_last_h)
  #print(encoder_last_c)

  encoder_last_h = BatchNormalization(momentum=0.6)(encoder_last_h)
  encoder_last_c = BatchNormalization(momentum=0.6)(encoder_last_c)
  decoder_input = RepeatVector(output_train.shape[1])(encoder_last_h)
  #print(decoder_input)

  decoder_stack_h = LSTM(200, activation='relu',dropout=0.2, recurrent_dropout=0.2,
                         kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None),
                        return_state=False, return_sequences=True)(decoder_input, initial_state=[encoder_last_h, encoder_last_c])

  attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
  attention = Activation('softmax')(attention)
  context = dot([attention, encoder_stack_h], axes=[2,1])
  context = BatchNormalization(momentum=0.6)(context)
  decoder_combined_context = concatenate([context, decoder_stack_h])
  out = TimeDistributed(Dense(output_train.shape[2]))(decoder_combined_context)
  model = Model(inputs=input_train, outputs=out)
  model.compile(optimizer='adam', loss='mean_absolute_error',metrics=["mse"])

  my_callbacks = [History(), EarlyStopping(patience=3, monitor="val_loss",restore_best_weights=True)]
  model.fit(X_train, Y_train, epochs=50, verbose=0,batch_size=64, validation_split=0.2, callbacks= my_callbacks)
  x_input = X_test.reshape((1, X_train.shape[1], 1))
  ypred = model.predict(x_input, verbose=0)
  y_pred= [np.round(x[0],1) for x in ypred[0]]
  return y_pred


In [ ]:
y_pred= atencionLuong(X_train, Y_train, X_test)
mean_squared_error(list(Y_test), y_pred)

7.938620888282549

## Comparación en validación

In [ ]:
%%time
data.shape

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 13.8 µs


(146, 3)

In [ ]:
%%time
largo_prediccion=[]
r_enc_dec=[]
r_att=[]
for i in range(data.shape[0]):
    Y_train= data[i][1]
    if Y_train.shape[1]> 1:
      largo_prediccion.append(Y_train.shape[1])
      X_train= data[i][0]
      Y_test= data[i][2]
      y_pred= enc_dec(X_train, Y_train, X_test)
      r_enc_dec.append(mean_squared_error(list(Y_test), y_pred))
      y_pred= atencionLuong(X_train, Y_train, X_test)
      r_att.append(mean_squared_error(list(Y_test), y_pred))
        
        

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f98abd77e60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


## caso many to one


In [ ]:
X_train= data[143][0]
Y_train= data[143][1]
Y_test= data[143][2]

In [ ]:
Y_train.shape

(346, 1)

In [ ]:
def biLSTM(X_traingo,Y_test,X_test,n_feautures=1):
  # define model
  model = Sequential()
  #enconder
  model.add(Bidirectional(LSTM(100, activation='relu',dropout=0.2, recurrent_dropout=0.2, activity_regularizer=tf.keras.regularizers.l1(0.01),
                kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None),input_shape=(X_train.shape[1], n_feautures) )))
  model.add(BatchNormalization())
  model.add(Dense(Y_train.shape[1])) #n_output

  model.compile(optimizer='adam', loss='mean_absolute_error',metrics=["mse"])

  my_callbacks = [History(), EarlyStopping(patience=3, monitor="val_loss",restore_best_weights=True)]
  model.fit(X_train, Y_train, epochs=300, verbose=0,batch_size=64, validation_split=0.2, callbacks= my_callbacks)

  x_input = X_test.reshape((1, X_train.shape[1], 1))
  ypred = model.predict(x_input, verbose=0)
  y_pred= np.round(ypred[0],1)
  return y_pred

In [ ]:
y_pred = biLSTM(X_train,Y_test,X_test)
mean_squared_error(list(Y_test), y_pred)

0.24999990463257782

In [ ]:
y_pred[0]

6.1

In [ ]:
largo_seq_predecir[largo_seq_predecir>5]

2015-06-28    29
2015-07-20    27
2015-06-15    27
2015-08-02    26
2015-05-28    25
              ..
2015-08-06     8
2015-08-21     7
2015-08-16     7
2015-08-18     6
2015-08-24     6
Length: 117, dtype: int64

In [ ]:
largo_seq_predecir[]

# Test

In [ ]:
horas_antes= 6*6 #6 horas
velocidad=list(train.Velocidad)
train["Fecha"] =  pd.to_datetime(train["Fecha"], format='%Y-%m-%d %H:%M')

dias_train_=train.Fecha.dt.date.unique()
str_fechas_entrenamiento= [str(x) for x in train.Fecha]


In [ ]:
X_test=np.array(velocidad[-horas_antes:]).reshape(1,horas_antes)

In [ ]:
test

,26-04-2015 01:50
0,26-04-2015 03:10
1,26-04-2015 04:20
2,26-04-2015 05:20
3,26-04-2015 06:40
4,26-04-2015 10:30
...,...
1994,19-09-2015 08:10
1995,21-09-2015 11:20
1996,22-09-2015 10:30
1997,23-09-2015 17:30


In [ ]:
fin_entrenamiento=datetime.strptime("26-04-2015", '%d-%m-%Y').date()

In [ ]:
Todos_los_X_Y_completo=[]
for i in range(largo_seq_predecir.shape[0]): #cada uno de los dias a predecir
  largo_pred= largo_seq_predecir[i]
  fecha_pred= largo_seq_predecir.index[i]
  dia_pred= fecha_pred.strftime("%Y-%m-%d")
  dia_sig=(fecha_pred +timedelta(days=1)).strftime("%Y-%m-%d")
  Y_test=l[(l>= dia_pred)&(l< dia_sig)]
  delta=(datetime.strptime(dia_pred, '%Y-%m-%d').date()-fin_entrenamiento).days
  horario_exacto=[]
  for dia in Y_test:
    horario_exacto.append(str(dia).split(" ")[1].split(":")[0]+":"+str(dia).split(" ")[1].split(":")[1])

  X_train,Y_train= generador_train_completo(largo_pred)
  
  Todos_los_X_Y_completo.append((X_train,Y_train))

In [ ]:
def generador_train_completo( largo_pred):
  i=0
  for d in dias_train_[1:]: #no tomo el primero
    y=[]
    for h in horario_exacto:
      consulta_y=str(d)+ " "+ h+":00"
      if consulta_y in str_fechas_entrenamiento:
        y.append(train[train["Fecha"]== str(d)+ " "+ h].Velocidad.values[0])
      else: 
        break 
    primer_y=train[train["Fecha"]==str(d)+ " "+ horario_exacto[0]].index[0]
    if len(y)==largo_pred and primer_y>horas_antes+120+delta*144:
      x=np.array(velocidad[primer_y-horas_antes-120-delta*144:primer_y-120-delta*144]).reshape(1,horas_antes)
      y= np.array(y).reshape(1,largo_pred)
      if i==0:
          i+=1
          X = x
          Y = y
      else:
          X = np.vstack((X,x))
          Y = np.vstack((Y,y))
    else:
      pass
  return X.reshape(X.shape[0], X.shape[1], 1), Y

In [ ]:
np.array(Todos_los_X_Y_completo)[0][0].shape

(792, 36, 1)

In [ ]:
save('train_completo.npy',np.array(Todos_los_X_Y_completo))

In [ ]:
Todos_los_X_Y_completo = np.load(path+"velocidad-viento/train_completo.npy",allow_pickle=True)

In [ ]:
Todos_los_X_Y_completo[145][1].shape

(711, 1)

## Evaluación

In [ ]:
ejemplo= pd.read_csv(path+'velocidad-viento/sampleSubmission.csv')

NameError: name 'path' is not defined

In [ ]:
ejemplo= pd.read_csv('velocidad-viento/sampleSubmission.csv')

In [ ]:
ejemplo[ejemplo.columns[1]]

,'Fecha','Velocidad de viento en 20.0 metros [mean]'
0,26-04-2015 01:50,5
1,26-04-2015 03:10,5
2,26-04-2015 04:20,4
3,26-04-2015 05:20,3
4,26-04-2015 06:40,3
...,...,...
1995,19-09-2015 08:10,1
1996,21-09-2015 11:20,4
1997,22-09-2015 10:30,3
1998,23-09-2015 17:30,11


In [ ]:
l_

0       26-04-2015 01:50
1       26-04-2015 03:10
2       26-04-2015 04:20
3       26-04-2015 05:20
4       26-04-2015 06:40
              ...       
1995    19-09-2015 08:10
1996    21-09-2015 11:20
1997    22-09-2015 10:30
1998    23-09-2015 17:30
1999    24-09-2015 00:10
Length: 2000, dtype: object

In [ ]:
test_date= [x.date() for x in l]

In [ ]:
lista_resultados=[]

In [ ]:
def completar_entregable(predicciones, fecha_pred):
  indices= indices_test(fecha_pred)

  for i in range(len(indices)):
    entregable[indices[i]]=  predicciones[i]
    lista_resultados.append(predicciones[i])


In [ ]:
def indices_test(fecha_pred):
  indices=[]
  for i in range(len(test_date)):
    f=test_date[i]
    if fecha_pred==f:
      indices.append(i)
  return indices

In [ ]:
evaluar = np.load(path+'velocidad-viento/train_completo.npy',allow_pickle=True)

In [ ]:
evaluar= np.array(Todos_los_X_Y_completo)

In [ ]:
for i in range(largo_seq_predecir.shape[0]): #cada uno de los dias a predecir
    largo_pred= largo_seq_predecir[i]
    fecha_pred= largo_seq_predecir.index[i]
    dia_pred= fecha_pred.strftime("%d-%m-%Y")
    X_train=evaluar[i][0]
    Y_train= evaluar[i][1]

    if largo_pred == 1: #lstm bidirectional
        predicciones=biLSTM(X_train,Y_test,X_test)
        completar_entregable(predicciones, fecha_pred)

    if largo_pred>=2: #atencion
        predicciones= atencionLuong(X_train, Y_train, X_test)
        completar_entregable(predicciones, fecha_pred)
        
    print(i) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145


In [ ]:
entregable

'Fecha'
26-04-2015 01:50     5.2
26-04-2015 03:10     4.8
26-04-2015 04:20     4.1
26-04-2015 05:20     3.4
26-04-2015 06:40     3.1
                    ... 
19-09-2015 08:10     1.8
21-09-2015 11:20     4.6
22-09-2015 10:30     4.2
23-09-2015 17:30    12.0
24-09-2015 00:10     7.1
Name: 'Velocidad de viento en 20.0 metros [mean]', Length: 2000, dtype: float64

In [ ]:
indices_test(fecha_pred)

[1544,
 1545,
 1546,
 1547,
 1548,
 1549,
 1550,
 1551,
 1552,
 1553,
 1554,
 1555,
 1556,
 1557,
 1558,
 1559,
 1560,
 1561,
 1562,
 1563,
 1564,
 1565,
 1566,
 1567,
 1568,
 1569,
 1570]

In [ ]:
def k():
    entregable[0]= 0.3223
    entregable[2]= 9999999

In [ ]:
k()

In [ ]:
entregable[1]

4.800000190734863

In [ ]:
def completar_entregable(predicciones, fecha_pred):
    indices= indices_test(fecha_pred)

    for i in range(len(indices)):
        entregable[indices[i]]=  predicciones[i]
        lista_resultados.append(predicciones[i])
    return entregable

In [ ]:
indices= indices_test(fecha_pred)

In [ ]:
indices

[1544,
 1545,
 1546,
 1547,
 1548,
 1549,
 1550,
 1551,
 1552,
 1553,
 1554,
 1555,
 1556,
 1557,
 1558,
 1559,
 1560,
 1561,
 1562,
 1563,
 1564,
 1565,
 1566,
 1567,
 1568,
 1569,
 1570]

In [ ]:
entregable[1554:1570]

'Fecha'
20-07-2015 07:00     5.8
20-07-2015 07:50     6.3
20-07-2015 10:00     6.8
20-07-2015 10:10     7.4
20-07-2015 11:20     8.0
20-07-2015 13:10     8.7
20-07-2015 16:50     9.4
20-07-2015 17:40    10.0
20-07-2015 18:40    10.6
20-07-2015 19:00    11.1
20-07-2015 20:00    11.4
20-07-2015 20:20    11.6
20-07-2015 20:50    11.6
20-07-2015 21:30    11.5
20-07-2015 22:50    11.2
20-07-2015 23:30    10.7
Name: 'Velocidad de viento en 20.0 metros [mean]', dtype: float64

In [ ]:
for i in range(len(indices)):
    entregable[indices[i]]=  predicciones[i]
    lista_resultados.append(predicciones[i])

In [ ]:
entregable[0]=  1.3

In [ ]:
entregable

'Fecha'
26-04-2015 01:50    1.3
26-04-2015 03:10    2.0
26-04-2015 04:20    0.0
26-04-2015 05:20    0.0
26-04-2015 06:40    0.0
                   ... 
19-09-2015 08:10    0.0
21-09-2015 11:20    0.0
22-09-2015 10:30    0.0
23-09-2015 17:30    0.0
24-09-2015 00:10    0.0
Name: 'Velocidad de viento en 20.0 metros [mean]', Length: 2000, dtype: float64

In [ ]:
entregable[1554:1570]

'Fecha'
20-07-2015 07:00    0.0
20-07-2015 07:50    0.0
20-07-2015 10:00    0.0
20-07-2015 10:10    0.0
20-07-2015 11:20    0.0
20-07-2015 13:10    0.0
20-07-2015 16:50    0.0
20-07-2015 17:40    0.0
20-07-2015 18:40    0.0
20-07-2015 19:00    0.0
20-07-2015 20:00    0.0
20-07-2015 20:20    0.0
20-07-2015 20:50    0.0
20-07-2015 21:30    0.0
20-07-2015 22:50    0.0
20-07-2015 23:30    0.0
Name: 'Velocidad de viento en 20.0 metros [mean]', dtype: float64

In [ ]:
entregable

'Fecha'
26-04-2015 01:50    0.0
26-04-2015 03:10    2.0
26-04-2015 04:20    0.0
26-04-2015 05:20    0.0
26-04-2015 06:40    0.0
                   ... 
19-09-2015 08:10    0.0
21-09-2015 11:20    0.0
22-09-2015 10:30    0.0
23-09-2015 17:30    0.0
24-09-2015 00:10    0.0
Name: 'Velocidad de viento en 20.0 metros [mean]', Length: 2000, dtype: float64

In [ ]:
entregable

'Fecha'
26-04-2015 01:50     5.2
26-04-2015 03:10     4.8
26-04-2015 04:20     4.1
26-04-2015 05:20     3.4
26-04-2015 06:40     3.1
                    ... 
19-09-2015 08:10     1.8
21-09-2015 11:20     4.6
22-09-2015 10:30     4.2
23-09-2015 17:30    12.0
24-09-2015 00:10     7.1
Name: 'Velocidad de viento en 20.0 metros [mean]', Length: 2000, dtype: float64

In [ ]:
float(ejemplo[ejemplo.columns[1]][1994])

2.0

In [ ]:
entregable_2.to_csv("sampleSubmission_mario_redondeado.csv")

In [ ]:
type(predicciones[0])

numpy.float32

In [ ]:
indices= indices_test(fecha_pred)

for i in range(len(indices)):
    print(predicciones[i])
    ejemplo[ejemplo.columns[1]][indices[i]]=  predicciones[i]

2.3


In [ ]:
indices[i]

1994

In [ ]:
ejemplo[ejemplo.columns[1]][indices[i]]=  predicciones[i]

In [ ]:
def vector(listatupla):
    summit=pd.DataFrame(listatupla)
    summit.columns=["'Fecha'","'Velocidad de viento en 20.0 metros [mean]'"]
    summit=summit.set_index("'Fecha'")
    summit= summit[summit.columns[0]].astype(float)
    return summit

In [ ]:
entregable=vector(ejemplo)

In [ ]:
copia= entregable.copy()

In [ ]:
entregable_2=np.round(entregable,1)

In [ ]:
entregable_2[0]

5.2

In [ ]:
for x in 

In [ ]:
entregable

'Fecha'
26-04-2015 01:50     5.2
26-04-2015 03:10     4.8
26-04-2015 04:20     4.1
26-04-2015 05:20     3.4
26-04-2015 06:40     3.1
                    ... 
19-09-2015 08:10     1.8
21-09-2015 11:20     4.6
22-09-2015 10:30     4.2
23-09-2015 17:30    12.0
24-09-2015 00:10     7.1
Name: 'Velocidad de viento en 20.0 metros [mean]', Length: 2000, dtype: float64

# 2) Idea Tendencias


In [ ]:
def preprocesamiento_sem(data, n_input, n_output, n_step):

  for i in range(0, data.shape[0], n_step):
    try:
      x = data[i:i+n_input].reshape(1,n_input)
      y = data[i+n_input: i+n_input+n_output].reshape(1,n_output)
      
      if i == 0:
        X = x
        Y = y
      else:
        X = np.vstack((X,x))
        Y = np.vstack((Y,y))
    except:
      pass

  return X.reshape(X.shape[0], X.shape[1], 1), Y

In [ ]:
def distancia_h(data, prom):
    k=[]
    for hora in range(144):
        r=[]
        for d in range(7):
            r.append(prom-data.iloc[d*144+hora]) 
        k.append(np.mean(r))
    return k
        

In [ ]:
i=train.shape[0]
prom_sem=[]
prom_d_horas=[]
while(i-7*144>0):
    data=train["Velocidad"][i-7*144:i]
    prom=data.mean()
    prom_sem.append(prom)
    prom_d_horas.append(distancia_h(data, prom))
    i-= 7*144
    
    
    
    

In [ ]:
len(prom_sem)

122

In [ ]:
len(prom_d_horas)

122

In [ ]:
X_train_s, Y_train_s = preprocesamiento_sem(np.array(prom_sem), 30, 22, 1)

In [ ]:
print(X_train_s.shape, Y_train_s.shape)

(71, 30, 1) (71, 22)


In [ ]:
X_val= X_train_s[-1]
Y_val=Y_train_s[-1]

X_train=X_train_s[:-1]
Y_train= Y_train_s[:-1]

In [ ]:
X_test_s= np.array(prom_sem[-30:]).reshape(1,30,1)

In [ ]:
X_test_s.shape

(1, 30, 1)

In [ ]:
# define model
model = Sequential()
#enconder
model.add(LSTM(100, activation='relu',
               kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None), input_shape=(X_train.shape[1], 1)))
model.add(RepeatVector(22))

#decoder
model.add(LSTM(100, activation='relu',
               kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None), return_sequences=True))
model.add(TimeDistributed(Dense(1)))

model.compile(optimizer='adam', loss='mse')
my_callbacks = [History(), EarlyStopping(patience=3, monitor="val_loss",restore_best_weights=True)]
history= model.fit(X_train_s, Y_train_s, epochs=300, verbose=1, batch_size=16,validation_split=0.2, callbacks= my_callbacks)

Epoch 1/300
4/4 [==============================] - 0s 74ms/step - loss: 54.6163 - val_loss: 36.6376
Epoch 2/300
4/4 [==============================] - 0s 15ms/step - loss: 23.6118 - val_loss: 10.2348
Epoch 3/300
4/4 [==============================] - 0s 14ms/step - loss: 9.8390 - val_loss: 6.4892
Epoch 4/300
4/4 [==============================] - 0s 14ms/step - loss: 6.6792 - val_loss: 5.2623
Epoch 5/300
4/4 [==============================] - 0s 14ms/step - loss: 4.8665 - val_loss: 4.0515
Epoch 6/300
4/4 [==============================] - 0s 14ms/step - loss: 3.8380 - val_loss: 3.9061
Epoch 7/300
4/4 [==============================] - 0s 14ms/step - loss: 3.5411 - val_loss: 3.2313
Epoch 8/300
4/4 [==============================] - 0s 14ms/step - loss: 2.9131 - val_loss: 2.9963
Epoch 9/300
4/4 [==============================] - 0s 13ms/step - loss: 2.5855 - val_loss: 2.9262
Epoch 10/300
4/4 [==============================] - 0s 13ms/step - loss: 2.3841 - val_loss: 2.7426
Epoch 11/300
4/

In [ ]:
x_input = X_val.reshape((1, X_train.shape[1], 1))
y_pred = model.predict(x_input, verbose=0)

In [ ]:
len(prom_sem)

122

In [ ]:
len(prom_d_horas)

122

In [ ]:
Y_horas= np.array(prom_d_horas).reshape(122,144)

In [ ]:
Y_horas.shape

(122, 144)

In [ ]:
def preprocesamiento_h(data, n_input, n_output, n_step):

  for i in range(0, data.shape[0], n_step):
    try:
      x = data[i:i+n_input].reshape(n_input,144)
      y = data[i+n_input: i+n_input+n_output].reshape(n_output,144)
      
      if i == 0:
        X = x
        Y = y
      else:
        X = np.vstack((X,x))
        Y = np.vstack((Y,y))
    except:
      pass

  return X.reshape(X.shape[0], X.shape[1], 1), Y

In [ ]:
X,Y= preprocesamiento_h(Y_horas,1,1,1)

In [ ]:
X.shape

(121, 144, 1)

In [ ]:
Y.shape

(121, 144)

In [ ]:
# define model
model_h = Sequential()
#enconder
model_h.add(LSTM(5, activation='relu',
               kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None), input_shape=(144, 1)))
model_h.add(Dense(144))

model_h.compile(optimizer='adam', loss='mse')

my_callbacks = [History(), EarlyStopping(patience=3, monitor="val_loss",restore_best_weights=True)]

history= model_h.fit(X, Y, epochs=300, verbose=1,validation_split=0.2, callbacks= my_callbacks)

Epoch 1/300
3/3 [==============================] - 0s 66ms/step - loss: 8.9338 - val_loss: 7.8138
Epoch 2/300
3/3 [==============================] - 0s 15ms/step - loss: 8.9123 - val_loss: 7.7932
Epoch 3/300
3/3 [==============================] - 0s 14ms/step - loss: 8.8908 - val_loss: 7.7730
Epoch 4/300
3/3 [==============================] - 0s 17ms/step - loss: 8.8702 - val_loss: 7.7527
Epoch 5/300
3/3 [==============================] - 0s 13ms/step - loss: 8.8488 - val_loss: 7.7322
Epoch 6/300
3/3 [==============================] - 0s 14ms/step - loss: 8.8278 - val_loss: 7.7113
Epoch 7/300
3/3 [==============================] - 0s 14ms/step - loss: 8.8066 - val_loss: 7.6900
Epoch 8/300
3/3 [==============================] - 0s 14ms/step - loss: 8.7838 - val_loss: 7.6682
Epoch 9/300
3/3 [==============================] - 0s 14ms/step - loss: 8.7615 - val_loss: 7.6455
Epoch 10/300
3/3 [==============================] - 0s 14ms/step - loss: 8.7380 - val_loss: 7.6219
Epoch 11/300
3/3 [=

In [ ]:
X[-1].shape

(144, 1)

In [ ]:
x_input = X[-2].reshape((1, 144, 1))
y_pred = model_h.predict(x_input, verbose=0)

# test

In [ ]:
152/7

21.714285714285715

In [ ]:
x_input = X_test_s.reshape((1,30, 1))
y_pred = model.predict(x_input, verbose=0)

In [ ]:
y_pred.shape

(1, 22, 1)

In [ ]:
x_input = X[-1].reshape((1, 144, 1))
y_pred_horas = model_h.predict(x_input, verbose=0)

In [ ]:
y_pred_horas.shape

(1, 144)

In [ ]:
ejemplo= pd.read_csv('velocidad-viento/sampleSubmission.csv')

In [ ]:
def vector(listatupla):
    summit=pd.DataFrame(listatupla)
    summit.columns=["'Fecha'","'Velocidad de viento en 20.0 metros [mean]'"]
    summit=summit.set_index("'Fecha'")
    summit= summit[summit.columns[0]].astype(float)
    return pd.DataFrame(summit)

In [ ]:
vector(ejemplo)

,'Velocidad de viento en 20.0 metros [mean]'
'Fecha',
26-04-2015 01:50,0.0
26-04-2015 03:10,0.0
26-04-2015 04:20,0.0
26-04-2015 05:20,0.0
26-04-2015 06:40,0.0
...,...
19-09-2015 08:10,0.0
21-09-2015 11:20,0.0
22-09-2015 10:30,0.0


In [ ]:
l

0      2015-04-26 01:50:00
1      2015-04-26 03:10:00
2      2015-04-26 04:20:00
3      2015-04-26 05:20:00
4      2015-04-26 06:40:00
               ...        
1995   2015-09-19 08:10:00
1996   2015-09-21 11:20:00
1997   2015-09-22 10:30:00
1998   2015-09-23 17:30:00
1999   2015-09-24 00:10:00
Length: 2000, dtype: datetime64[ns]

In [ ]:
ejemplo

,'Fecha','Velocidad de viento en 20.0 metros [mean]'
0,26-04-2015 01:50,0
1,26-04-2015 03:10,0
2,26-04-2015 04:20,0
3,26-04-2015 05:20,0
4,26-04-2015 06:40,0
...,...,...
1995,19-09-2015 08:10,0
1996,21-09-2015 11:20,0
1997,22-09-2015 10:30,0
1998,23-09-2015 17:30,0


In [ ]:
l[0].date()

datetime.date(2015, 4, 26)

In [ ]:
l[100].date()

datetime.date(2015, 5, 1)

In [ ]:
a=l[0].date()-l[100].date()
a.days

-5

In [ ]:
l

0      2015-04-26 01:50:00
1      2015-04-26 03:10:00
2      2015-04-26 04:20:00
3      2015-04-26 05:20:00
4      2015-04-26 06:40:00
               ...        
1995   2015-09-19 08:10:00
1996   2015-09-21 11:20:00
1997   2015-09-22 10:30:00
1998   2015-09-23 17:30:00
1999   2015-09-24 00:10:00
Length: 2000, dtype: datetime64[ns]

In [ ]:
y_pred[0][0]

array([7.067201], dtype=float32)

In [ ]:
l[0].time()

datetime.time(1, 50)

In [ ]:
int(l[0].time().hour*6+ (l[0].time().minute)/10)

11

In [ ]:
y_pred_horas[0][2]

0.5442344

In [ ]:
l[10].date()-l[10-1].date()

datetime.timedelta(0)

In [ ]:
l

0      2015-04-26 01:50:00
1      2015-04-26 03:10:00
2      2015-04-26 04:20:00
3      2015-04-26 05:20:00
4      2015-04-26 06:40:00
               ...        
1995   2015-09-19 08:10:00
1996   2015-09-21 11:20:00
1997   2015-09-22 10:30:00
1998   2015-09-23 17:30:00
1999   2015-09-24 00:10:00
Length: 2000, dtype: datetime64[ns]

In [ ]:
(l[100].date()-l[0].date()).days

5

In [ ]:
l[0]

Timestamp('2015-04-26 01:50:00')

In [ ]:
l[0]+timedelta(minutes= 10)

Timestamp('2015-04-26 02:00:00')

In [ ]:
pd.to_datetime("26-04-2015 00:00",format='%d-%m-%Y %H:%M')== pd.to_datetime("26-04-2015 00:10",format='%d-%m-%Y %H:%M')

False

In [ ]:
timedelta(minutes= 10)

datetime.timedelta(seconds=600)

In [ ]:
l

0      2015-04-26 01:50:00
1      2015-04-26 03:10:00
2      2015-04-26 04:20:00
3      2015-04-26 05:20:00
4      2015-04-26 06:40:00
               ...        
1995   2015-09-19 08:10:00
1996   2015-09-21 11:20:00
1997   2015-09-22 10:30:00
1998   2015-09-23 17:30:00
1999   2015-09-24 00:10:00
Length: 2000, dtype: datetime64[ns]

In [ ]:
fecha= pd.to_datetime("26-04-2015 00:00",format='%d-%m-%Y %H:%M')
fechas_completas=[]
while(fecha != pd.to_datetime("24-09-2015 00:20",format='%d-%m-%Y %H:%M')):
    fechas_completas.append(fecha)
    fecha= fecha+timedelta(minutes= 10)
    
    
    

In [ ]:
resultado=[]
for semana in  y_pred[0]:
    for dia in range(7):
        for hora in y_pred_horas[0]:
            resultado.append(float(semana)+hora)    

In [ ]:
len(resultado)

21746

In [ ]:
len(fechas_completas)

21746

In [ ]:
resultado= resultado[:len(fechas_completas)]

In [ ]:
l

0      2015-04-26 01:50:00
1      2015-04-26 03:10:00
2      2015-04-26 04:20:00
3      2015-04-26 05:20:00
4      2015-04-26 06:40:00
               ...        
1995   2015-09-19 08:10:00
1996   2015-09-21 11:20:00
1997   2015-09-22 10:30:00
1998   2015-09-23 17:30:00
1999   2015-09-24 00:10:00
Length: 2000, dtype: datetime64[ns]

In [ ]:
resultado[11]

array([8.301934], dtype=float32)

In [ ]:
fechas_completas.index(l[0])

11

In [ ]:
resultado

[array([7.595862], dtype=float32),
 array([7.620867], dtype=float32),
 array([7.6114354], dtype=float32),
 array([7.7179713], dtype=float32),
 array([7.806505], dtype=float32),
 array([7.966244], dtype=float32),
 array([7.971353], dtype=float32),
 array([8.020852], dtype=float32),
 array([8.1463], dtype=float32),
 array([8.136526], dtype=float32),
 array([8.370214], dtype=float32),
 array([8.301934], dtype=float32),
 array([8.500356], dtype=float32),
 array([8.432432], dtype=float32),
 array([8.498327], dtype=float32),
 array([8.514758], dtype=float32),
 array([8.540367], dtype=float32),
 array([8.713871], dtype=float32),
 array([8.857152], dtype=float32),
 array([8.798668], dtype=float32),
 array([8.891548], dtype=float32),
 array([8.942765], dtype=float32),
 array([9.057807], dtype=float32),
 array([9.076251], dtype=float32),
 array([9.156215], dtype=float32),
 array([9.258865], dtype=float32),
 array([8.52165], dtype=float32),
 array([9.293065], dtype=float32),
 array([9.40056], dty

In [ ]:
lista_vacia=[]
for fecha in l:
    ind=fechas_completas.index(fecha)
    lista_vacia.append((fecha, resultado[ind]))  

In [ ]:
submit=vector(pd.DataFrame(lista_vacia))

In [ ]:
submit.to_csv("sampleSubmission_tendencias.csv")

In [ ]:
submit.index= submit.index.strftime("%d-%m-%Y %H:%M")

In [ ]:
submit

,'Velocidad de viento en 20.0 metros [mean]'
'Fecha',
26-04-2015 01:50,8.301934
26-04-2015 03:10,8.798668
26-04-2015 04:20,8.521651
26-04-2015 05:20,9.640955
26-04-2015 06:40,9.801782
...,...
19-09-2015 08:10,10.687980
21-09-2015 11:20,8.121248
22-09-2015 10:30,8.859510


In [ ]:
train.iloc[93743]

Fecha        2014-10-01 00:00:00
Velocidad                    4.1
Name: 93743, dtype: object

In [ ]:
train.iloc[71002]

Fecha        2014-04-26 01:50:00
Velocidad                   10.1
Name: 71002, dtype: object

In [ ]:
train.iloc[71002:93743].mean()

Velocidad    8.013324
dtype: float64

In [ ]:
pred= pd.read_csv(path+'velocidad-viento/sampleSubmission_mario_redondeado.csv')

In [ ]:
pred.mean()

'Velocidad de viento en 20.0 metros [mean]'    7.4277
dtype: float64

In [ ]:
pred.std()

'Velocidad de viento en 20.0 metros [mean]'    2.968269
dtype: float64

In [ ]:
pred[pred.columns[1]]

0        5.2
1        4.8
2        4.1
3        3.4
4        3.1
        ... 
1995     1.8
1996     4.6
1997     4.2
1998    12.0
1999     7.1
Name: 'Velocidad de viento en 20.0 metros [mean]', Length: 2000, dtype: float64

In [ ]:
p= np.array(pred[pred.columns[1]])

In [ ]:
(p+0.5).mean()

7.927700000000001

In [ ]:
otrosum=vector(p+0.5)


ValueError: ignored

In [ ]:
pred[pred.columns[1]]= pred[pred.columns[1]]+0.5

In [ ]:
vector(pred)

,'Velocidad de viento en 20.0 metros [mean]'
'Fecha',
26-04-2015 01:50,5.7
26-04-2015 03:10,5.3
26-04-2015 04:20,4.6
26-04-2015 05:20,3.9
26-04-2015 06:40,3.6
...,...
19-09-2015 08:10,2.3
21-09-2015 11:20,5.1
22-09-2015 10:30,4.7


In [ ]:
vector(pred).to_csv("modelo_mario+05.csv")